# 全球大気モデル１５０年連続実験のハンドリングと可視化

気候予測データセット2022には、
気象研究所大気大循環モデルMRI-AGCM3.2に海面水温・海氷被覆データを境界条件として与え、過去の気候を計算し、
さらに将来の気候を計算した、150年連続実験の計算結果が含まれている。

150年連続実験では、
空間解像度60kmの大気モデルに対して、過去再現実験では４種類の海面水温・海氷被覆データを与えた計算、
将来はRCPの4シナリオを与えて計算が行われた。

|過去|将来|
|:-|:-|
|HPD|HFD_rcp26|
|HPD_m01|HFD_rcp45|
|HPD_m02|HFD_rcp60|
|HPD_m03|HFD_rcp85|

さらにRCP8.5を与えた空間解像度60kmのAGCMの計算結果を、空間解像度20kmのAGCMでダウンスケーリングされている。

DIASに格納されている計算結果で、今回みることができるデータは、
* HPD : 過去再現実験。
* HPD_m01 : 過去再現実験。SSTが若干異なる計算。
* HPD_m02 : 過去再現実験。SSTが若干異なる計算。
* HPD_m03 : 過去再現実験。SSTが若干異なる計算。
* HFD_HighResMIP : RCP8.5シナリオでAGCM60→AGCM20でダウンロードした計算。
* HDF_rcp26 : RCP2.6シナリオの計算結果。
* HDF_rcp45 : RCP4.5シナリオの計算結果。
* HDF_rcp60 : RCP460シナリオの計算結果。

である。
細かな計算条件などは、気候予測データセット2022の解説書第２章で確認してください。

このハンズオンセミナーでは、
月平均の地上気象要素のファイル（sfc_avr_mon_HPD_*.dr）を例としてデータの処理・可視化作業を行う。

なお、データのハンドリングについては、**気候変化情報研究室**による
[「全球及び日本域150年連続実験データ」を可視化する その４ー統合プログラム全球60km150年連続実験データを入手しQGIS上でアニメーション表示する](https://cci-labo.hateblo.jp/entry/2023/08/29/120411)
の手法を利用する。

なお、[xgrads](https://github.com/miniufo/xgrads) を用いることでいわゆる「GrADS形式（単純なバイナリーファイル）」を
xarray.Datasetに読み込むことができるはずである。
しかし、pipやcondaでインストールされるxgradsでは、150年連続実験のファイルのフォルダの構成に対応していない（2024年10月後半時点）。

ライブラリの読み込み。

In [1]:
import os
import pathlib
import datetime
import numpy as np
import pandas as pd
import xarray as xr

左側のFile Browserの"dias-data"→"GCM60_NHRCM20_150yr_TOUGOU"→"GCM60"→"HPD"と移動する。

そして、GrADSのctlファイルを左側のFile Browserの検索窓で探す。
検索窓に「sfc_avr_mon.ctl」と打ち込み、検索結果のファイルを右クリックし、"Copy Path"をする。
コピーしたパスを次のように設定する。

In [2]:
ctlFile = "dias-data/GCM60_NHRCM20_150yr_TOUGOU/GCM60/HPD/sfc_avr_mon.ctl"

GrADSのctlファイルからYDEFの情報が含まれる位置、変数の情報が含まれる位置を取得する。

In [3]:
substr_df = pd.read_fwf(
    ctlFile,
    widths=[10],
    header=None,
    skip_blank_lines=False
).fillna("#")

In [4]:
YDEF_ind = substr_df.loc[substr_df[0].str.startswith("YDEF")].index.values[0] 

In [5]:
YDEF_nrow = substr_df.loc[substr_df[0].str.startswith("ZDEF")].index.values[0] - YDEF_ind - 1

In [6]:
VARS_ind = substr_df.loc[substr_df[0].str.startswith("VARS")].index.values[0]

In [7]:
VARS_nrow = substr_df.loc[substr_df[0].str.startswith("ENDVARS")].index.values[0] - VARS_ind - 1

AGCMの緯度方向の座標の情報を取得する。

In [8]:
lat = pd.read_fwf(
    ctlFile,
    header=None,
    skiprows=YDEF_ind+1,
    nrows=YDEF_nrow,
    colspecs=[(6,16),(17,27),(28,38),(39,49),(50,60)]
).stack().values

AGCMの経度方向の座標を作成する。

In [9]:
lon = np.arange(640) * 0.56250

変数の情報を取得する。

In [10]:
parm_df = pd.read_fwf(
    ctlFile,
    header=None,
    skiprows=VARS_ind+1,
    nrows=VARS_nrow,
    colspecs=[(0,9),(25,100),(90,104),(107,108)],
    names=["varname","description","unit","flag"]
)
parm_df

,varname,description,unit,flag
0,TA,Surface Air Temperature at 2m ...,K,1
1,TGEF,Effective Ground temperature (Radiation) ...,K,1
2,SLP,SEA LEVEL PRESSSURE ...,Pa,1
3,PS,SURFACE PRESSURE ...,Pa,1
4,UA,Surface Zonal Velocity at 10m ...,m/s,1
5,VA,Surface Merid. Velocity at 10m ...,m/s,1
6,WIND,Surface Air Wind Speed at 10m ...,m/s,1
7,RHA,Surface Air Relative Humidity at 2m ...,%,1
8,QA,Surface Air Specific Humidity at 2m ...,kg/kg,1
9,PRECIPI,Total Precipitation ...,kg/m**2/s,1


ファイルのディレクトリを``bdir``に設定し、
``pathlib``と``glob``を利用してファイルの一覧を取得しデータフレームに格納する。

In [11]:
#dias-data/GCM60_NHRCM20_150yr_TOUGOU/GCM60/HPD/195001/sfc_avr_mon_HPD_195001.dr
bdir = "dias-data/GCM60_NHRCM20_150yr_TOUGOU/GCM60/HPD/"
drlist_df = pd.DataFrame({"dr_path":list(pathlib.Path(bdir).glob("**/sfc_avr_mon_HPD_1950*.dr"))})
drlist_df

,dr_path
0,dias-data/GCM60_NHRCM20_150yr_TOUGOU/GCM60/HPD...
1,dias-data/GCM60_NHRCM20_150yr_TOUGOU/GCM60/HPD...
2,dias-data/GCM60_NHRCM20_150yr_TOUGOU/GCM60/HPD...
3,dias-data/GCM60_NHRCM20_150yr_TOUGOU/GCM60/HPD...
4,dias-data/GCM60_NHRCM20_150yr_TOUGOU/GCM60/HPD...
5,dias-data/GCM60_NHRCM20_150yr_TOUGOU/GCM60/HPD...
6,dias-data/GCM60_NHRCM20_150yr_TOUGOU/GCM60/HPD...
7,dias-data/GCM60_NHRCM20_150yr_TOUGOU/GCM60/HPD...
8,dias-data/GCM60_NHRCM20_150yr_TOUGOU/GCM60/HPD...
9,dias-data/GCM60_NHRCM20_150yr_TOUGOU/GCM60/HPD...


データフレームのファイルのパスからファイル名だけを抽出し、"dr_name"というカラムに格納する。

In [12]:
drlist_df["dr_name"] = drlist_df["dr_path"].map(lambda x: x.name)
drlist_df

,dr_path,dr_name
0,dias-data/GCM60_NHRCM20_150yr_TOUGOU/GCM60/HPD...,sfc_avr_mon_HPD_195001.dr
1,dias-data/GCM60_NHRCM20_150yr_TOUGOU/GCM60/HPD...,sfc_avr_mon_HPD_195002.dr
2,dias-data/GCM60_NHRCM20_150yr_TOUGOU/GCM60/HPD...,sfc_avr_mon_HPD_195003.dr
3,dias-data/GCM60_NHRCM20_150yr_TOUGOU/GCM60/HPD...,sfc_avr_mon_HPD_195004.dr
4,dias-data/GCM60_NHRCM20_150yr_TOUGOU/GCM60/HPD...,sfc_avr_mon_HPD_195005.dr
5,dias-data/GCM60_NHRCM20_150yr_TOUGOU/GCM60/HPD...,sfc_avr_mon_HPD_195006.dr
6,dias-data/GCM60_NHRCM20_150yr_TOUGOU/GCM60/HPD...,sfc_avr_mon_HPD_195007.dr
7,dias-data/GCM60_NHRCM20_150yr_TOUGOU/GCM60/HPD...,sfc_avr_mon_HPD_195008.dr
8,dias-data/GCM60_NHRCM20_150yr_TOUGOU/GCM60/HPD...,sfc_avr_mon_HPD_195009.dr
9,dias-data/GCM60_NHRCM20_150yr_TOUGOU/GCM60/HPD...,sfc_avr_mon_HPD_195010.dr


データフレームのファイル名から年・月の除法を抽出し、"date"というカラムに格納する。

In [13]:
drlist_df["date"] = pd.to_datetime(drlist_df["dr_name"].str.extract(".*_(\d{6}).dr")[0],format="%Y%m")
drlist_df

,dr_path,dr_name,date
0,dias-data/GCM60_NHRCM20_150yr_TOUGOU/GCM60/HPD...,sfc_avr_mon_HPD_195001.dr,1950-01-01
1,dias-data/GCM60_NHRCM20_150yr_TOUGOU/GCM60/HPD...,sfc_avr_mon_HPD_195002.dr,1950-02-01
2,dias-data/GCM60_NHRCM20_150yr_TOUGOU/GCM60/HPD...,sfc_avr_mon_HPD_195003.dr,1950-03-01
3,dias-data/GCM60_NHRCM20_150yr_TOUGOU/GCM60/HPD...,sfc_avr_mon_HPD_195004.dr,1950-04-01
4,dias-data/GCM60_NHRCM20_150yr_TOUGOU/GCM60/HPD...,sfc_avr_mon_HPD_195005.dr,1950-05-01
5,dias-data/GCM60_NHRCM20_150yr_TOUGOU/GCM60/HPD...,sfc_avr_mon_HPD_195006.dr,1950-06-01
6,dias-data/GCM60_NHRCM20_150yr_TOUGOU/GCM60/HPD...,sfc_avr_mon_HPD_195007.dr,1950-07-01
7,dias-data/GCM60_NHRCM20_150yr_TOUGOU/GCM60/HPD...,sfc_avr_mon_HPD_195008.dr,1950-08-01
8,dias-data/GCM60_NHRCM20_150yr_TOUGOU/GCM60/HPD...,sfc_avr_mon_HPD_195009.dr,1950-09-01
9,dias-data/GCM60_NHRCM20_150yr_TOUGOU/GCM60/HPD...,sfc_avr_mon_HPD_195010.dr,1950-10-01


　"date"行で日付によるソートを行う。

In [14]:
drlist_df = drlist_df.sort_values("date").reset_index(drop=True)
drlist_df

,dr_path,dr_name,date
0,dias-data/GCM60_NHRCM20_150yr_TOUGOU/GCM60/HPD...,sfc_avr_mon_HPD_195001.dr,1950-01-01
1,dias-data/GCM60_NHRCM20_150yr_TOUGOU/GCM60/HPD...,sfc_avr_mon_HPD_195002.dr,1950-02-01
2,dias-data/GCM60_NHRCM20_150yr_TOUGOU/GCM60/HPD...,sfc_avr_mon_HPD_195003.dr,1950-03-01
3,dias-data/GCM60_NHRCM20_150yr_TOUGOU/GCM60/HPD...,sfc_avr_mon_HPD_195004.dr,1950-04-01
4,dias-data/GCM60_NHRCM20_150yr_TOUGOU/GCM60/HPD...,sfc_avr_mon_HPD_195005.dr,1950-05-01
5,dias-data/GCM60_NHRCM20_150yr_TOUGOU/GCM60/HPD...,sfc_avr_mon_HPD_195006.dr,1950-06-01
6,dias-data/GCM60_NHRCM20_150yr_TOUGOU/GCM60/HPD...,sfc_avr_mon_HPD_195007.dr,1950-07-01
7,dias-data/GCM60_NHRCM20_150yr_TOUGOU/GCM60/HPD...,sfc_avr_mon_HPD_195008.dr,1950-08-01
8,dias-data/GCM60_NHRCM20_150yr_TOUGOU/GCM60/HPD...,sfc_avr_mon_HPD_195009.dr,1950-09-01
9,dias-data/GCM60_NHRCM20_150yr_TOUGOU/GCM60/HPD...,sfc_avr_mon_HPD_195010.dr,1950-10-01


一時的な配列を作成する。１年分の地上気温のデータを格納するため、経度方向640、緯度方向320、時間方向12の大きさの配列である。

In [15]:
tmpAry = np.zeros([12,320,640],np.float32)

それぞれのファイルを``numpy.fromfile``を用いて"mon_dat"に読み込み、
``reshape``して"mon_arr"に代入する。
気温のデータは59種類の変数のうち一番最初のデータなので、"mon_arr[0,:,:]"が気温のデータとなる。
気温のデータを"tmpAry[h_index,:,:]"に代入する。
なお、h_indexは（月数-1）を意味する。

In [16]:
for h_index,h_row in drlist_df.iterrows():
    mon_dat = np.fromfile(str(h_row["dr_path"]),dtype='>f4')
    mon_arr = mon_dat.reshape(59,320,640)
    tmpAry[h_index,:,:] = mon_arr[0,:,:]

一時的な配列tmpAryを``xarray.Dataset``に格納する。

In [17]:
ds = xr.Dataset(
    {
        "tas": (["time", "lat", "lon" ],tmpAry)
    },
    coords={
        "lat": ("lat", lat),
        "lon": ("lon", lon),
        "time": drlist_df["date"],
    },
)
ds

<xarray.Dataset> Size: 10MB
Dimensions:  (time: 12, lat: 320, lon: 640)
Coordinates:
  * lat      (lat) float64 3kB -89.57 -89.01 -88.45 -87.89 ... 88.45 89.01 89.57
  * lon      (lon) float64 5kB 0.0 0.5625 1.125 1.688 ... 358.3 358.9 359.4
  * time     (time) datetime64[ns] 96B 1950-01-01 1950-02-01 ... 1950-12-01
Data variables:
    tas      (time, lat, lon) float32 10MB 246.6 246.6 246.6 ... 245.8 245.8

それぞれの変数に属性（attribution）を、できるだけ「**CF規約**」に従うように付してゆく。

In [18]:
ds['lon'] = ds['lon'].assign_attrs(
    units = 'degrees_east',
    long_name = 'longitude',
    axis = 'X'
)
ds['lat'] = ds['lat'].assign_attrs(
    units = 'degrees_north',
    long_name = 'latitude',
    axis = 'Y'
)
ds['time'] = ds['time'].assign_attrs(
    axis = 'T'
)
ds['tas'] = ds['tas'].assign_attrs(
    standard_name = 'air_temperature',
    long_name = 'Near Surface Air Temperature',
    units = 'K',
    _FillValue = -9.99E33
)
ds

<xarray.Dataset> Size: 10MB
Dimensions:  (time: 12, lat: 320, lon: 640)
Coordinates:
  * lat      (lat) float64 3kB -89.57 -89.01 -88.45 -87.89 ... 88.45 89.01 89.57
  * lon      (lon) float64 5kB 0.0 0.5625 1.125 1.688 ... 358.3 358.9 359.4
  * time     (time) datetime64[ns] 96B 1950-01-01 1950-02-01 ... 1950-12-01
Data variables:
    tas      (time, lat, lon) float32 10MB 246.6 246.6 246.6 ... 245.8 245.8

dsをnetcdf形式のファイルに格納する。
時刻の情報は``xarray``では``datetime64型``であるが、
netcdf形式で保管する際には reference となる日付・時刻からの差に変換して格納する。
そのためにencodingの中で'time'に'units'と'dtype'を設定する。

In [19]:
ds.to_netcdf(
    "./tmp.ex1.raw.nc",
    encoding = {
        'time': {
            'units': 'days since 1950-01-01 00:00:00',
            'dtype': 'float64'
            },
        'lat': {'_FillValue': None},
        'lon': {'_FillValue': None}
    },
    unlimited_dims = 'time'
)

netcdf4では変数を圧縮して格納することができる。
変数を圧縮する際にはencodingの中の'tas'の行のように記述する。

In [20]:
ds.to_netcdf(
    "./tmp1.ex1.zipped.nc",
    encoding = {
        'time': {
            'units': 'days since 1950-01-01 00:00:00',
            'dtype': 'float64'
            },
        'lat': {'_FillValue': None},
        'lon': {'_FillValue': None},
        'tas': {'zlib': True, 'complevel': 4}
    },
    unlimited_dims = 'time'
)

In [21]:
del ds

150年連続実験の過去再現実験HPDの地上気温データを抽出し、ファイルに格納する。

In [22]:
#dias-data/GCM60_NHRCM20_150yr_TOUGOU/GCM60/HPD/195001/sfc_avr_mon_HPD_195001.dr
bdir = "dias-data/GCM60_NHRCM20_150yr_TOUGOU/GCM60/HPD/"
for yyyy in ["195", "196", "197", "198", "199", "200", "201"]:
    str1 = "**/sfc_avr_mon_HPD_"+yyyy+"*.dr"
    print(str1)
    drlist_df = pd.DataFrame({"dr_path":list(pathlib.Path(bdir).glob(str1))})
    drlist_df["dr_name"] = drlist_df["dr_path"].map(lambda x: x.name)
    drlist_df["date"] = pd.to_datetime(drlist_df["dr_name"].str.extract(".*_(\d{6}).dr")[0],format="%Y%m")
    drlist_df = drlist_df.sort_values("date").reset_index(drop=True)
    numFiles = len(drlist_df)
    tmpAry = np.zeros([numFiles,320,640],np.float32)
    for h_index,h_row in drlist_df.iterrows():
        mon_dat = np.fromfile(str(h_row["dr_path"]),dtype='>f4')
        mon_arr = mon_dat.reshape(59,320,640)
        tmpAry[h_index,:,:] = mon_arr[0,:,:]
    ds = xr.Dataset(
        {
            "tas": (["time", "lat", "lon" ],tmpAry)
        },
        coords={
            "lat": ("lat", lat),
            "lon": ("lon", lon),
            "time": drlist_df["date"],},
    )
    ds['lon'] = ds['lon'].assign_attrs(
        units = 'degrees_east',long_name = 'longitude',axis = 'X'
    )
    ds['lat'] = ds['lat'].assign_attrs(
        units = 'degrees_north',long_name = 'latitude',axis = 'Y'
    )
    ds['time'] = ds['time'].assign_attrs(
        axis = 'T'
    )
    ds['tas'] = ds['tas'].assign_attrs(
        standard_name = 'air_temperature',
        long_name = 'Near Surface Air Temperature',
        units = 'K',
        _FillValue = -9.99E33
    )
    if yyyy == "201":
        nameOut = "./tas."+yyyy+"0-"+yyyy+"4.nc"
    else:
        nameOut = "./tas."+yyyy+"0-"+yyyy+"9.nc"
    print(nameOut)
    dt = datetime.datetime.now()
    dtStr = dt.isoformat()
    ds.assign_attrs(
        title = "MRI-AGCM HFD historical tas",
        convention = "None",
        history = dtStr + "test_AGCM.ipynb",
    )
    ds.to_netcdf(
        nameOut,
        encoding = {
            'time': {
                'units': 'days since 1950-01-01 00:00:00',
                'dtype': 'float64'
            },
            'lat': {'_FillValue': None},
            'lon': {'_FillValue': None},
            'tas': {'zlib': True, 'complevel': 4}
        },
        unlimited_dims = 'time'
    )
    del ds


**/sfc_avr_mon_HPD_195*.dr
./tas.1950-1959.nc
**/sfc_avr_mon_HPD_196*.dr
./tas.1960-1969.nc
**/sfc_avr_mon_HPD_197*.dr
./tas.1970-1979.nc
**/sfc_avr_mon_HPD_198*.dr
./tas.1980-1989.nc
**/sfc_avr_mon_HPD_199*.dr
./tas.1990-1999.nc
**/sfc_avr_mon_HPD_200*.dr
./tas.2000-2009.nc
**/sfc_avr_mon_HPD_201*.dr
./tas.2010-2014.nc


cdoのmergetime機能を利用して、年代ごとのファイルを一つにまとめる。

In [23]:
!cdo mergetime tas.*.nc tas.HFD.1950-2014.nc

cdo    mergetime: Processed 159744000 values from 7 variables over 780 timesteps [5.99s 93MB].
